# 📊 Análisis de Segmentación de Clientes con RFM y Machine Learning

## 🎯 Objetivo del Proyecto

Este notebook demuestra cómo usar **SQL**, **Python** y **Machine Learning** para segmentar clientes de un e-commerce en grupos accionables utilizando la metodología **RFM (Recencia, Frecuencia, Monetario)** y el algoritmo **K-Means**.

### ¿Por qué es importante?
- **Recencia (R)**: ¿Cuán recientemente compró el cliente?
- **Frecuencia (F)**: ¿Cuántas veces ha comprado?
- **Monetario (M)**: ¿Cuánto dinero ha gastado?

Combinando estas métricas, identificaremos:
- 🏆 **Clientes Campeones**: Los mejores clientes, compradores frecuentes y recientes
- ⚠️ **Clientes en Riesgo**: Que no compran desde hace tiempo
- 💪 **Clientes Leales**: Compradores consistentes
- 🆕 **Clientes Nuevos**: Nuevas adquisiciones

### 📚 Librerías Utilizadas
- `sqlite3`: Gestión de base de datos SQL
- `pandas`: Manipulación de datos
- `numpy`: Operaciones numéricas
- `sklearn`: Machine Learning (K-Means, StandardScaler)
- `matplotlib` & `seaborn`: Visualización de datos

In [ ]:
# 1. IMPORTAR LIBRERÍAS NECESARIAS
# ═════════════════════════════════════════════════════════════════════════════

import pandas as pd
import numpy as np
import sqlite3
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

# Librerías de Machine Learning
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar estilo de gráficos
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Todas las librerías importadas correctamente")

## Sección 1: Crear y Cargar Datos en Base de Datos SQLite

En esta sección, **cargamos datos CSV en una base de datos SQL**. Esto simula un entorno real donde los datos están en una BD en lugar de archivos CSV.

### ¿Por qué es importante?
- Aprendes a trabajar con **bases de datos reales** (como un verdadero analista)
- Practicas **SQL** para consultar datos
- Simulas un pipeline real: Datos → BD → Análisis

In [ ]:
# 2. CARGAR DATOS Y LIMPIARLOS
# ═════════════════════════════════════════════════════════════════════════════

print("📂 Cargando datos del archivo CSV...")

# Cargar el CSV del proyecto anterior (usando ruta relativa)
df = pd.read_csv('../project1/data.csv', encoding='latin1')

print(f"✅ Datos cargados: {df.shape[0]} filas, {df.shape[1]} columnas")
print(f"\nColumnas disponibles:\n{df.columns.tolist()}")
print(f"\nPrimeras 5 filas:")
print(df.head())

In [ ]:
# 3. LIMPIAR Y PREPARAR DATOS
# ═════════════════════════════════════════════════════════════════════════════

print("\n🧹 Limpiando datos...")

# Convertir InvoiceDate a datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='%m/%d/%Y %H:%M')

# Eliminar filas con valores críticos faltantes
df = df.dropna(subset=['CustomerID', 'InvoiceNo', 'Country'])

# Filtrar transacciones válidas (cantidad y precio positivos)
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

# Crear columna de precio total
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

print(f"✅ Datos limpios: {df.shape[0]} filas válidas")
print(f"📊 Período de datos: {df['InvoiceDate'].min()} a {df['InvoiceDate'].max()}")
print(f"👥 Clientes únicos: {df['CustomerID'].nunique()}")
print(f"💰 Ingresos totales: ${df['TotalPrice'].sum():,.2f}")

In [ ]:
# 4. CREAR BASE DE DATOS SQLITE
# ═════════════════════════════════════════════════════════════════════════════

print("\n🗄️  Creando base de datos SQLite...")

# Crear conexión a la base de datos (la crea si no existe)
conn = sqlite3.connect('ecommerce.db')

# Cargar el DataFrame en la tabla 'transacciones'
df.to_sql('transacciones', conn, if_exists='replace', index=False)

print("✅ Base de datos 'ecommerce.db' creada exitosamente")
print("✅ Tabla 'transacciones' cargada con datos")

# Verificar que la tabla se creó correctamente
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM transacciones")
row_count = cursor.fetchone()[0]
print(f"📊 Total de registros en BD: {row_count}")

cursor.execute("SELECT COUNT(DISTINCT CustomerID) FROM transacciones")
customer_count = cursor.fetchone()[0]
print(f"👥 Total de clientes únicos en BD: {customer_count}")

conn.close()
print("\n✅ Conexión cerrada. Base de datos lista para consultas SQL.")

## Sección 2: Calcular Métricas RFM usando SQL

Ahora conectaremos a nuestra base de datos y usaremos **SQL puro** para calcular:
- **Recencia**: Días desde la última compra
- **Frecuencia**: Cantidad de transacciones
- **Monetario**: Valor total gastado

Esta es la parte clave: **escribimos SQL desde Python**. Así trabajan los analistas reales.

In [ ]:
# 5. CONSULTAR BASE DE DATOS CON SQL
# ═════════════════════════════════════════════════════════════════════════════

print("\n🔍 Consultando base de datos con SQL...")

# Conectar a la base de datos
conn = sqlite3.connect('ecommerce.db')

# CONSULTA SQL 1: Obtener métricas RFM base
# Esto es SQL puro, ejecutado directamente en la base de datos
sql_rfm = """
SELECT
    CustomerID,
    MAX(InvoiceDate) AS UltimaFechaCompra,
    COUNT(DISTINCT InvoiceNo) AS Frecuencia,
    SUM(TotalPrice) AS Monetario
FROM 
    transacciones
GROUP BY 
    CustomerID
ORDER BY 
    Monetario DESC
"""

# Ejecutar la consulta usando pandas (pandas se conecta a SQLite)
df_rfm_base = pd.read_sql_query(sql_rfm, conn)

print(f"✅ Consulta SQL ejecutada correctamente")
print(f"📊 Resultados: {df_rfm_base.shape[0]} clientes con métricas RFM")
print(f"\nPrimeros 10 clientes (ordenados por Monetario):")
print(df_rfm_base.head(10))

# Cerrar conexión
conn.close()

In [ ]:
# 6. CALCULAR RECENCIA EN PANDAS
# ═════════════════════════════════════════════════════════════════════════════

print("\n⏰ Calculando Recencia (días desde última compra)...")

# Convertir UltimaFechaCompra a datetime
df_rfm_base['UltimaFechaCompra'] = pd.to_datetime(df_rfm_base['UltimaFechaCompra'])

# Definir "hoy" como un día después de la última compra en el dataset
fecha_maxima = df_rfm_base['UltimaFechaCompra'].max()
hoy = fecha_maxima + dt.timedelta(days=1)

# Calcular recencia en días
df_rfm_base['Recencia'] = (hoy - df_rfm_base['UltimaFechaCompra']).dt.days

print(f"📅 Fecha máxima en datos: {fecha_maxima.date()}")
print(f"📅 Fecha 'hoy' usada: {hoy.date()}")
print(f"⏳ Recencia mínima: {df_rfm_base['Recencia'].min()} días")
print(f"⏳ Recencia máxima: {df_rfm_base['Recencia'].max()} días")
print(f"⏳ Recencia promedio: {df_rfm_base['Recencia'].mean():.0f} días")

# Crear DataFrame final con RFM
df_rfm = df_rfm_base[['CustomerID', 'Recencia', 'Frecuencia', 'Monetario']].copy()

print(f"\n✅ DataFrame RFM completado:")
print(df_rfm.describe())

## Sección 3: Preprocesar Datos para Machine Learning

El algoritmo K-Means es sensible a la escala de los datos:
- Recencia: 0-200 días
- Frecuencia: 0-200 compras
- Monetario: 0-100,000 USD

Si usamos estos valores directamente, Monetario domina el algoritmo.

**Solución**: 
1. Aplicar transformación logarítmica (log) para normalizar distribuciones sesgadas
2. Usar StandardScaler para estandarizar todas las características a la misma escala

In [ ]:
# 7. PREPROCESAR DATOS PARA MACHINE LEARNING
# ═════════════════════════════════════════════════════════════════════════════

print("\n🔧 Preprocesando datos para K-Means...")

# Paso 1: Aplicar transformación logarítmica
# Esto ayuda a normalizar distribuciones sesgadas
df_rfm_log = df_rfm[['Recencia', 'Frecuencia', 'Monetario']].copy()
df_rfm_log = df_rfm_log.apply(lambda x: np.log1p(x), axis=1)  # log1p = log(1+x)

print("✅ Transformación logarítmica aplicada")
print(f"\nEstadísticas después de log transform:")
print(df_rfm_log.describe())

# Paso 2: Estandarizar los datos (StandardScaler)
# Esto pone todos los datos en la misma escala (media=0, desv.est=1)
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(df_rfm_log)

print(f"\n✅ StandardScaler aplicado")
print(f"Forma de datos escalados: {rfm_scaled.shape}")
print(f"Media de datos escalados: {rfm_scaled.mean(axis=0)}")
print(f"Desv. estándar escalados: {rfm_scaled.std(axis=0)}")

## Sección 4: Aplicar K-Means Clustering

K-Means es un algoritmo que:
1. Divide los datos en **k clusters** (grupos)
2. Cada cluster tiene un **centroide** (punto central)
3. Cada cliente es asignado al cluster más cercano

Para este proyecto, usaremos **4 clusters** porque identificamos 4 tipos de clientes:
- Campeones
- En Riesgo
- Leales
- Nuevos

In [ ]:
# 8. ENTRENAR MODELO K-MEANS
# ═════════════════════════════════════════════════════════════════════════════

print("\n🤖 Entrenando modelo K-Means...")

# Crear y entrenar el modelo K-Means con 4 clusters
# init='k-means++': inicialización inteligente de centroides
# random_state=42: reproducibilidad
# n_init=10: probar 10 inicializaciones diferentes
kmeans = KMeans(
    n_clusters=4,
    init='k-means++',
    random_state=42,
    n_init=10,
    max_iter=300
)

# Entrenar el modelo
kmeans.fit(rfm_scaled)

print("✅ Modelo K-Means entrenado")
print(f"📊 Inercia (suma de distancias intra-cluster): {kmeans.inertia_:.2f}")

# Calcular Silhouette Score (métrica de calidad del clustering)
# Rango: -1 a 1, mientras más cerca a 1, mejor
silhouette_avg = silhouette_score(rfm_scaled, kmeans.labels_)
print(f"🎯 Silhouette Score: {silhouette_avg:.3f}")

# Asignar etiquetas de cluster a los clientes
df_rfm['Segmento'] = kmeans.labels_

print(f"\n✅ Clusters asignados a todos los clientes")
print(f"📊 Distribución de clientes por cluster:")
print(df_rfm['Segmento'].value_counts().sort_index())

## Sección 5: Analizar e Interpretar Segmentos

Ahora interpretaremos cada cluster basándonos en sus características RFM:
- **Recencia BAJA + Frecuencia ALTA + Monetario ALTO** → Clientes Campeones 🏆
- **Recencia ALTA + Frecuencia BAJA + Monetario BAJO** → Clientes en Riesgo ⚠️
- **Recencia MEDIA + Frecuencia MEDIA + Monetario MEDIO** → Clientes Leales 💪
- **Recencia BAJA + Frecuencia BAJA + Monetario BAJO** → Clientes Nuevos 🆕

In [ ]:
# 9. ANALIZAR CARACTERÍSTICAS DE CADA SEGMENTO
# ═════════════════════════════════════════════════════════════════════════════

print("\n📊 Analizando características de cada segmento...")

# Calcular estadísticas por segmento
segmentos_resumen = df_rfm.groupby('Segmento').agg({
    'Recencia': ['mean', 'min', 'max'],
    'Frecuencia': ['mean', 'min', 'max'],
    'Monetario': ['mean', 'min', 'max'],
    'CustomerID': 'count'
}).round(2)

# Aplanar los nombres de columnas para mejor legibilidad
segmentos_resumen.columns = ['_'.join(col).strip() for col in segmentos_resumen.columns.values]
segmentos_resumen = segmentos_resumen.rename(columns={'CustomerID_count': 'Total_Clientes'})

print("\n✅ Resumen de segmentos:")
print(segmentos_resumen)

# Crear un DataFrame más limpio para análisis
segmentos_clean = df_rfm.groupby('Segmento').agg({
    'Recencia': 'mean',
    'Frecuencia': 'mean',
    'Monetario': 'mean',
    'CustomerID': 'count'
}).round(2)
segmentos_clean.columns = ['Recencia_Media', 'Frecuencia_Media', 'Monetario_Media', 'Total_Clientes']
segmentos_clean['% del Total'] = (segmentos_clean['Total_Clientes'] / segmentos_clean['Total_Clientes'].sum() * 100).round(1)

print("\n📈 Resumen Simplificado:")
print(segmentos_clean)

In [ ]:
# 10. ASIGNAR NOMBRES A LOS SEGMENTOS
# ═════════════════════════════════════════════════════════════════════════════

print("\n🏷️  Interpretando segmentos basado en características RFM...")

# Basándose en el análisis anterior, crear mapeo de clusters a nombres
# ¡IMPORTANTE!: Este mapeo debe ajustarse según los datos reales
# Para este ejemplo, usamos la lógica RFM estándar

def asignar_nombre_segmento(row):
    """
    Asigna un nombre significativo a cada segmento basado en sus características RFM
    """
    recencia = row['Recencia_Media']
    frecuencia = row['Frecuencia_Media']
    monetario = row['Monetario_Media']
    
    # Calcular percentiles para comparación
    recencia_percentil = recencia / df_rfm['Recencia'].max()
    frecuencia_percentil = frecuencia / df_rfm['Frecuencia'].max()
    monetario_percentil = monetario / df_rfm['Monetario'].max()
    
    # Lógica de clasificación
    if frecuencia_percentil >= 0.6 and monetario_percentil >= 0.6 and recencia_percentil <= 0.4:
        return 'Campeones 🏆'
    elif recencia_percentil >= 0.6 and frecuencia_percentil <= 0.4 and monetario_percentil <= 0.4:
        return 'En Riesgo ⚠️'
    elif frecuencia_percentil >= 0.5 and monetario_percentil >= 0.5:
        return 'Leales 💪'
    elif recencia_percentil <= 0.3 and frecuencia_percentil <= 0.4:
        return 'Nuevos 🆕'
    else:
        return 'Otros'

# Aplicar la función para asignar nombres
segmentos_clean['Nombre'] = segmentos_clean.apply(asignar_nombre_segmento, axis=1)

# Agregar el nombre al DataFrame principal
mapa_nombres = dict(zip(segmentos_clean.index, segmentos_clean['Nombre']))
df_rfm['NombreSegmento'] = df_rfm['Segmento'].map(mapa_nombres)

print("✅ Nombres asignados a segmentos")
print("\n📌 Mapeo de Segmentos:")
for seg, nombre in mapa_nombres.items():
    count = (df_rfm['Segmento'] == seg).sum()
    pct = (count / len(df_rfm) * 100)
    print(f"   Cluster {seg}: {nombre} - {count} clientes ({pct:.1f}%)")

## Sección 6: Visualizar Resultados de Segmentación

Ahora crearemos gráficos para visualizar los segmentos y entender mejor las características de cada grupo.

In [ ]:
# 11. VISUALIZACIONES DE SEGMENTOS
# ═════════════════════════════════════════════════════════════════════════════

print("\n📊 Creando visualizaciones...")

# Crear figura con múltiples subgráficos
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Análisis de Segmentación de Clientes RFM', fontsize=16, fontweight='bold')

# Gráfico 1: Distribución de clientes por segmento (Pie Chart)
ax1 = axes[0, 0]
colores = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A']
segmento_counts = df_rfm['NombreSegmento'].value_counts()
ax1.pie(segmento_counts.values, labels=segmento_counts.index, autopct='%1.1f%%', colors=colores, startangle=90)
ax1.set_title('Distribución de Clientes por Segmento', fontweight='bold')

# Gráfico 2: Recencia vs Monetario (Scatter)
ax2 = axes[0, 1]
scatter = ax2.scatter(df_rfm['Recencia'], df_rfm['Monetario'], 
                     c=df_rfm['Segmento'], cmap='viridis', alpha=0.6, s=50)
ax2.set_xlabel('Recencia (días)', fontweight='bold')
ax2.set_ylabel('Monetario (USD)', fontweight='bold')
ax2.set_title('Recencia vs Monetario por Segmento', fontweight='bold')
ax2.grid(True, alpha=0.3)
plt.colorbar(scatter, ax=ax2, label='Segmento')

# Gráfico 3: Métrica promedio por segmento (Bar Chart)
ax3 = axes[1, 0]
segmentos_metricas = df_rfm.groupby('NombreSegmento')[['Recencia', 'Frecuencia', 'Monetario']].mean()
segmentos_metricas.plot(kind='bar', ax=ax3, color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
ax3.set_title('Métricas Promedio por Segmento', fontweight='bold')
ax3.set_ylabel('Valor Promedio', fontweight='bold')
ax3.set_xlabel('Segmento', fontweight='bold')
ax3.legend(title='Métrica', loc='best')
ax3.tick_params(axis='x', rotation=45)
ax3.grid(True, alpha=0.3, axis='y')

# Gráfico 4: Frecuencia vs Monetario
ax4 = axes[1, 1]
for segmento in df_rfm['Segmento'].unique():
    datos = df_rfm[df_rfm['Segmento'] == segmento]
    ax4.scatter(datos['Frecuencia'], datos['Monetario'], 
               label=mapa_nombres[segmento], alpha=0.6, s=50)
ax4.set_xlabel('Frecuencia (# compras)', fontweight='bold')
ax4.set_ylabel('Monetario (USD)', fontweight='bold')
ax4.set_title('Frecuencia vs Monetario', fontweight='bold')
ax4.legend(loc='best')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Visualizaciones completadas")

## Sección 7: Recomendaciones de Negocio

Basado en el análisis RFM, aquí están las estrategias accionables para cada segmento:

In [ ]:
# 12. GENERAR RECOMENDACIONES DE NEGOCIO
# ═════════════════════════════════════════════════════════════════════════════

print("\n" + "="*80)
print("🎯 RECOMENDACIONES DE NEGOCIO POR SEGMENTO")
print("="*80)

recomendaciones = {
    'Campeones 🏆': {
        'descripcion': 'Clientes de alto valor que compran frecuentemente y recientemente',
        'acciones': [
            '✅ Programa VIP exclusivo con descuentos y beneficios especiales',
            '✅ Acceso anticipado a nuevos productos',
            '✅ Soporte premium y atención personalizada',
            '✅ Recompensas por lealtad (puntos, regalos)',
            '✅ Invitaciones a eventos especiales'
        ],
        'objetivo': 'Retener a los mejores clientes y maximizar su lifetime value'
    },
    'En Riesgo ⚠️': {
        'descripcion': 'Clientes que no compran hace tiempo, bajo valor, baja frecuencia',
        'acciones': [
            '✅ Campaña de "Te extrañamos" con descuento atractivo',
            '✅ Email marketing personalizado con ofertas relevantes',
            '✅ Encuesta de satisfacción para entender por qué dejaron de comprar',
            '✅ Ofertas de reactivación con urgencia (tiempo limitado)',
            '✅ Recordatorios periódicos sin ser invasivo'
        ],
        'objetivo': 'Reactivar clientes y evitar que se conviertan en inactivos'
    },
    'Leales 💪': {
        'descripcion': 'Clientes con compras consistentes, valor moderado',
        'acciones': [
            '✅ Programa de lealtad con puntos acumulables',
            '✅ Compras cruzadas recomendadas (cross-sell)',
            '✅ Venta adicional de productos premium (upsell)',
            '✅ Contenido exclusivo y educativo',
            '✅ Comunidad de clientes con beneficios'
        ],
        'objetivo': 'Incrementar valor de compra y convertirlos en Campeones'
    },
    'Nuevos 🆕': {
        'descripcion': 'Clientes recientes, valor bajo, baja frecuencia',
        'acciones': [
            '✅ Bienvenida personalizada con guía de productos',
            '✅ Descuento o cupón para segunda compra',
            '✅ Tutorial de uso de plataforma (si aplica)',
            '✅ Encuesta de experiencia para mejorar',
            '✅ Emails educativos sobre productos populares'
        ],
        'objetivo': 'Convertir en clientes recurrentes y aumentar frecuencia'
    }
}

for segmento, info in recomendaciones.items():
    print(f"\n{segmento}")
    print("-" * 80)
    print(f"📌 Descripción: {info['descripcion']}")
    print(f"📌 Objetivo: {info['objetivo']}")
    print(f"\n📋 Acciones recomendadas:")
    for accion in info['acciones']:
        print(f"   {accion}")

print("\n" + "="*80)

In [ ]:
# 13. EXPORTAR RESULTADOS Y CREAR ARCHIVOS FINALES
# ═════════════════════════════════════════════════════════════════════════════

print("\n💾 Exportando resultados...")

# Exportar clientes segmentados a CSV
df_rfm.to_csv('clientes_segmentados.csv', index=False)
print("✅ Archivo 'clientes_segmentados.csv' generado")

# Exportar resumen de segmentos
segmentos_clean.to_csv('resumen_segmentos.csv')
print("✅ Archivo 'resumen_segmentos.csv' generado")

# Exportar modelo KMeans (opcional)
import pickle
with open('kmeans_model.pkl', 'wb') as f:
    pickle.dump({'model': kmeans, 'scaler': scaler}, f)
print("✅ Modelo K-Means guardado en 'kmeans_model.pkl'")

print("\n" + "="*80)
print("📊 RESUMEN FINAL DEL PROYECTO")
print("="*80)
print(f"\n✅ Base de datos SQLite creada: 'ecommerce.db'")
print(f"✅ Clientes analizados: {len(df_rfm):,}")
print(f"✅ Segmentos identificados: 4")
print(f"✅ Silhouette Score: {silhouette_avg:.3f}")
print(f"\n📁 Archivos generados:")
print(f"   • clientes_segmentados.csv")
print(f"   • resumen_segmentos.csv")
print(f"   • kmeans_model.pkl")
print(f"   • ecommerce.db")
print(f"\n📊 Distribución de segmentos:")
for nombre, count in df_rfm['NombreSegmento'].value_counts().items():
    pct = (count / len(df_rfm) * 100)
    print(f"   {nombre}: {count:,} clientes ({pct:.1f}%)")
print("\n" + "="*80)
print("🎉 ¡Proyecto RFM completado exitosamente!")
print("="*80)